The following code creates a dataframe and loads a list of folders in the project column. Please download and extract the dataset yourself and adapt the paths if needed. 
https://github.com/lin-tan/llm-vul

In [383]:
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
import pandas as pd
import os

df = pd.DataFrame()

df=pd.DataFrame(os.listdir("llm-vul-main\llm-vul-main\VJBench-trans"),columns=['project'])

In [384]:
df

,project
0,BC-Java-1
1,Flow-1
2,Flow-2
3,Halo-1
4,Jenkins-1
5,Jenkins-2
6,Jenkins-3
7,Jinjava-1
8,Json-sanitizer-1
9,Netty-1


This code adds the code of the original method. The dataset has additional files with partial transformations. Maybe, they are more useful for us, feel free to modify this.

In [385]:
df=pd.DataFrame(os.listdir("llm-vul-main\llm-vul-main\VJBench-trans"),columns=['project'])
df['original_method']=df.apply(lambda x: open(os.path.join("llm-vul-main\llm-vul-main\VJBench-trans",x['project']+"\\"+x['project']+"_original_method.java"),'r').read(), axis=1)

In [386]:
df

,project,original_method
0,BC-Java-1,"private BigInteger[] derDecode(\n byte[] encoding)\n throws IOException\n{\n ASN1Sequence s = (ASN1Sequence)ASN1Primitive.fromByteArray(encoding);\n return new BigInteger[]{\n ((ASN1Integer)s.getObjectAt(0)).getValue(),\n ((ASN1Integer)s.getObjectAt(1)).getValue()\n };\n}\n"
1,Flow-1,"public boolean serveDevModeRequest(HttpServletRequest request,\n HttpServletResponse response) throws IOException {\n if (isDevServerFailedToStart.get() || !devServerStartFuture.isDone()) {\n return false;\n }\n String requestFilename = request.getPathInfo();\n if (HandlerHelper.isPathUnsafe(requestFilename)) {\n getLogger().info(HandlerHelper.UNSAFE_PATH_ERROR_MESSAGE_PATTERN,\n requestFilename);\n response.setStatus(HttpServletResponse.SC_FORBIDDEN);\n return true;\n }\n if(APP_THEME_PATTERN.matcher(requestFilename).find()) {\n requestFilename = ""/VAADIN/static"" + requestFilename;\n }\n HttpURLConnection connection = prepareConnection(requestFilename,\n request.getMethod());\n Enumeration<String> headerNames = request.getHeaderNames();\n while (headerNames.hasMoreElements()) {\n String header = headerNames.nextElement();\n connection.setRequestProperty(header,\n ""Connect"".equals(header) ? ""close""\n : request.getHeader(header));\n }\n getLogger().debug(""Requesting resource to webpack {}"",\n connection.getURL());\n int responseCode = connection.getResponseCode();\n if (responseCode == HTTP_NOT_FOUND) {\n getLogger().debug(""Resource not served by webpack {}"",\n requestFilename);\n return false;\n }\n getLogger().debug(""Served resource by webpack: {} {}"", responseCode,\n requestFilename);\n connection.getHeaderFields().forEach((header, values) -> {\n if (header != null) {\n response.addHeader(header, values.get(0));\n }\n });\n if (responseCode == HTTP_OK) {\n writeStream(response.getOutputStream(),\n connection.getInputStream());\n } else if (responseCode < 400) {\n response.setStatus(responseCode);\n } else {\n response.sendError(responseCode);\n }\n response.getOutputStream().close();\n return true;\n}\n"
2,Flow-2,"@Override\npublic int setErrorParameter(BeforeEnterEvent event,\n ErrorParameter<NotFoundException> parameter) {\n String path = event.getLocation().getPath();\n String additionalInfo = """";\n if (parameter.hasCustomMessage()) {\n additionalInfo = ""Reason: "" + parameter.getCustomMessage();\n }\n path = Jsoup.clean(path, Whitelist.none());\n additionalInfo = Jsoup.clean(additionalInfo, Whitelist.none());\n boolean productionMode = event.getUI().getSession().getConfiguration()\n .isProductionMode();\n String template = getErrorHtml(productionMode);\n template = template.replace(""{{path}}"", path);\n template = template.replace(""{{additionalInfo}}"", additionalInfo);\n if (template.contains(""{{routes}}"")) {\n template = template.replace(""{{routes}}"", getRoutes(event));\n }\n\n getElement().appendChild(new Html(template).getElement());\n return HttpServletResponse.SC_NOT_FOUND;\n}\n"
3,Halo-1,"public static void checkDirectoryTraversal(@NonNull Path parentPath, @NonNull Path pathToCheck) {\n Assert.notNull(parentPath, ""Parent path must not be null"");\n Assert.notNull(pathToCheck, ""Path to check must not be null"");\n if (pathToCheck.startsWith(parentPath.normalize())) {\n return;\n }\n throw new ForbiddenException(""You do not have permission to access "" + pathToCheck).setErrorData(pathToCheck);\n}\n"
4,Jenkins-1,"@Exported(inline=true)\npublic Map<String,Object> getMonitorData() {\n Map<String,Object> r = new HashMap<String, Object>();\n for (NodeMonitor monitor : NodeMonitor.getAll())\n r.put(monitor.getClass().getName(),monitor.data(this));\n return r;\n}\n"
5,Jenkins-2,"@Exported(name=""jobs"")\npublic List<TopLevelItem> getItems() {\n if (authorizationStrategy instanceof AuthorizationStrategy.Unsecured ||\n authorizationStrategy instanceof FullControlOnceLoggedInAuthorizationStrategy) {\n return new ArrayList(items.values());\n }\n\n List<TopLevelItem> viewableItems = new

This adds the location of the bug in the original method. It is a list with 2 elements (start and end line); usually the same. 

In [387]:
import json
df['location_original_method']=df.apply(lambda x:json.load(open(os.path.join("llm-vul-main\llm-vul-main\VJBench-trans",x['project']+"\\"+"buggyline_location.json")))['original'],axis=1)

The InputFeatures class copied from the open source project

In [388]:
class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 input_tokens,
                 input_ids,
                 label):
        self.input_tokens = input_tokens
        self.input_ids = input_ids
        self.label=label

Loading the model and tokeniser and move them to gpu if available. I have downloaded the model and this is a local path. Feel free to use the model id if it works for you. I am using codebert-base. 

In [389]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = RobertaTokenizer.from_pretrained("./model")
model = RobertaModel.from_pretrained("./model")
model.to(device)


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

The actual tokenization happens below. Maybe think about padding. Last week we discussed that we do not need to pad to the max line length

In [390]:
block_size=10


In [391]:
df

,project,original_method,location_original_method
0,BC-Java-1,"private BigInteger[] derDecode(\n byte[] encoding)\n throws IOException\n{\n ASN1Sequence s = (ASN1Sequence)ASN1Primitive.fromByteArray(encoding);\n return new BigInteger[]{\n ((ASN1Integer)s.getObjectAt(0)).getValue(),\n ((ASN1Integer)s.getObjectAt(1)).getValue()\n };\n}\n","[[6, 6]]"
1,Flow-1,"public boolean serveDevModeRequest(HttpServletRequest request,\n HttpServletResponse response) throws IOException {\n if (isDevServerFailedToStart.get() || !devServerStartFuture.isDone()) {\n return false;\n }\n String requestFilename = request.getPathInfo();\n if (HandlerHelper.isPathUnsafe(requestFilename)) {\n getLogger().info(HandlerHelper.UNSAFE_PATH_ERROR_MESSAGE_PATTERN,\n requestFilename);\n response.setStatus(HttpServletResponse.SC_FORBIDDEN);\n return true;\n }\n if(APP_THEME_PATTERN.matcher(requestFilename).find()) {\n requestFilename = ""/VAADIN/static"" + requestFilename;\n }\n HttpURLConnection connection = prepareConnection(requestFilename,\n request.getMethod());\n Enumeration<String> headerNames = request.getHeaderNames();\n while (headerNames.hasMoreElements()) {\n String header = headerNames.nextElement();\n connection.setRequestProperty(header,\n ""Connect"".equals(header) ? ""close""\n : request.getHeader(header));\n }\n getLogger().debug(""Requesting resource to webpack {}"",\n connection.getURL());\n int responseCode = connection.getResponseCode();\n if (responseCode == HTTP_NOT_FOUND) {\n getLogger().debug(""Resource not served by webpack {}"",\n requestFilename);\n return false;\n }\n getLogger().debug(""Served resource by webpack: {} {}"", responseCode,\n requestFilename);\n connection.getHeaderFields().forEach((header, values) -> {\n if (header != null) {\n response.addHeader(header, values.get(0));\n }\n });\n if (responseCode == HTTP_OK) {\n writeStream(response.getOutputStream(),\n connection.getInputStream());\n } else if (responseCode < 400) {\n response.setStatus(responseCode);\n } else {\n response.sendError(responseCode);\n }\n response.getOutputStream().close();\n return true;\n}\n","[[7, 8]]"
2,Flow-2,"@Override\npublic int setErrorParameter(BeforeEnterEvent event,\n ErrorParameter<NotFoundException> parameter) {\n String path = event.getLocation().getPath();\n String additionalInfo = """";\n if (parameter.hasCustomMessage()) {\n additionalInfo = ""Reason: "" + parameter.getCustomMessage();\n }\n path = Jsoup.clean(path, Whitelist.none());\n additionalInfo = Jsoup.clean(additionalInfo, Whitelist.none());\n boolean productionMode = event.getUI().getSession().getConfiguration()\n .isProductionMode();\n String template = getErrorHtml(productionMode);\n template = template.replace(""{{path}}"", path);\n template = template.replace(""{{additionalInfo}}"", additionalInfo);\n if (template.contains(""{{routes}}"")) {\n template = template.replace(""{{routes}}"", getRoutes(event));\n }\n\n getElement().appendChild(new Html(template).getElement());\n return HttpServletResponse.SC_NOT_FOUND;\n}\n","[[14, 18]]"
3,Halo-1,"public static void checkDirectoryTraversal(@NonNull Path parentPath, @NonNull Path pathToCheck) {\n Assert.notNull(parentPath, ""Parent path must not be null"");\n Assert.notNull(pathToCheck, ""Path to check must not be null"");\n if (pathToCheck.startsWith(parentPath.normalize())) {\n return;\n }\n throw new ForbiddenException(""You do not have permission to access "" + pathToCheck).setErrorData(pathToCheck);\n}\n","[[4, 4]]"
4,Jenkins-1,"@Exported(inline=true)\npublic Map<String,Object> getMonitorData() {\n Map<String,Object> r = new HashMap<String, Object>();\n for (NodeMonitor monitor : NodeMonitor.getAll())\n r.put(monitor.getClass().getName(),monitor.data(this));\n return r;\n}\n","[[4, 5]]"
5,Jenkins-2,"@Exported(name=""jobs"")\npublic List<TopLevelItem> getItems() {\n if (authorizationStrategy instanceof AuthorizationStrategy.Unsecured ||\n authorizationStrategy instanceof FullControlOnceLoggedInAuthorizationStrategy) {\n re

In [392]:
examples = []
def line_to_features(line, target):
    line=line.strip()#remove any whitespaces before or after the line
    code_tokens = tokenizer.tokenize(str(line))[:block_size-2]
    source_tokens = [tokenizer.cls_token] + code_tokens + [tokenizer.sep_token]
    padding_length = block_size - len(source_tokens)
    source_tokens += [tokenizer.pad_token] * padding_length
    source_ids = tokenizer.convert_tokens_to_ids(source_tokens)
    padding_length = block_size - len(source_ids)
    source_ids += [tokenizer.pad_token_id] * padding_length
    return InputFeatures(source_tokens, source_ids, target)


def function_to_features(function, target_lines):
    #return [line_to_features(line,True ) for ind,line in enumerate(function.split('\n')) if line.strip()!='' ] 
    return [line_to_features(line,target_lines[0][0]>=ind and target_lines[0][1]<=ind ) for ind,line in enumerate(function.split('\n')) if line.strip()!='' ] 
#examples=df.apply(lambda x:examples.append (function_to_features(x['original_method'],x['location_original_method'])), axis=1)#actually run anything

df['input_features']=df.apply(lambda function: function_to_features(function['original_method'],function['location_original_method']), axis=1)

In [393]:
examples=[item for row in df['input_features'] for item in row]

The following can be used for train/test split

In [394]:
examples

 ...]

In [395]:
examples[1].input_tokens

['<s>', 'byte', '[]', 'Ġ', 'Ġencoding', ')', '</s>', '<pad>', '<pad>', '<pad>']

In [396]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(examples, test_size=0.1, random_state=42)

In [397]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipe = Pipeline(
    steps=[
        #("scaler", StandardScaler()),
        #("knn", KNeighborsClassifier(n_neighbors=11))]
        ("rfc",     RandomForestClassifier(
            max_depth=5, n_estimators=10, max_features=1, random_state=42
        ))]
)


In [398]:
pipe

Pipeline(steps=[('rfc',
                 RandomForestClassifier(max_depth=5, max_features=1,
                                        n_estimators=10, random_state=42))])

In [399]:
train_x=list(map(lambda line:line.input_ids,train))
train_y=list(map(lambda line:line.label,train))

In [400]:
pipe.fit(train_x, train_y)

Pipeline(steps=[('rfc',
                 RandomForestClassifier(max_depth=5, max_features=1,
                                        n_estimators=10, random_state=42))])

In [401]:
len(train_x[0])

10

In [402]:
len(train_x[1])

10

In [403]:
test_x=list(map(lambda line:line.input_ids,test))
test_y=list(map(lambda line:line.label,test))

In [404]:
r=pipe.predict(train_x)
r

array([False, False, False, ..., False, False, False])

In [405]:
pipe.predict(test_x)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [406]:
test_y

[False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 Fals